In [ ]:
import cv2
import numpy as np
import glob
import time
import math
from skimage import io, color, filters, measure
import scipy.ndimage as ndi
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
import matplotlib.pyplot as plt
import os
import csv
from sklearn.cluster import KMeans


In [ ]:
#SEGMENTATION

# Function to perform image segmentation
def perform_segmentation(image_path):
    # Load the image and convert it to grayscale
    img = io.imread(image_path)
    img_gray = color.rgb2gray(img)

    # Compute the distance transform
    distance = ndi.distance_transform_edt(img_gray)

    # Calculate the threshold using Otsu's method
    threshold = filters.threshold_otsu(img_gray)

    # Find peaks in the distance map
    markers = peak_local_max(distance, min_distance=10, labels=img_gray > threshold)

    # Resize markers to the same shape as the mask
    markers = np.resize(markers, img_gray.shape)

    # Perform watershed segmentation
    labeled_image = watershed(-distance, markers, mask=img_gray)

    return labeled_image

# Function to extract features from segmented regions
def extract_features_from_segmentation(labeled_image, img_gray, label, filepath):
    # Calculate the features from each segmented region
    regions = measure.regionprops(labeled_image, intensity_image=img_gray)
    features = []
    concave_points_values = []
    symmetry_values = []
    fractal_dimension_values = []

    count = 0
    for i, region in enumerate(regions):
        # print("region", region, region.perimeter, region.area, region.convex_area, region.convex_image)
        if region.area == 0 or region.perimeter == 0:
            continue
        radius = region.perimeter / (2.0 * np.pi)
        texture = np.std(img_gray[region.slice])
        perimeter = region.perimeter
        area = region.area
        smoothness = (perimeter ** 2) / area
        compactness = (perimeter ** 2) / area
        concavity = region.convex_area - area
        concave_points = len(region.convex_image) - np.sum(region.convex_image)
        IMG_SIZE = max(img_gray.shape)
        symmetry = np.sum(np.abs(img_gray[region.slice] - np.flipud(img_gray[region.slice]))) / (IMG_SIZE ** 2)
        fractal_dimension = np.log(4 * np.pi * area) / (2 * np.log(perimeter))

        if concave_points != 0:
            concave_points_values.append(concave_points)
        if concave_points == 0:
            concave_points = np.mean(concave_points_values)

        if symmetry != 0:
            symmetry_values.append(symmetry)
        if symmetry == 0:
            symmetry = np.mean(symmetry_values)

        if fractal_dimension != 0:
            fractal_dimension_values.append(fractal_dimension)
        if fractal_dimension == 0:
            fractal_dimension = np.mean(fractal_dimension_values)

        count = count+1
        feature = {
            'radius': radius,
            'texture': texture,
            'perimeter': perimeter,
            'area': area,
            'smoothness': smoothness,
            'compactness': compactness,
            'concavity': concavity,
            'concave_points': concave_points,
            'symmetry': symmetry,
            'fractal_dimension': fractal_dimension,
            'label': label,
            'path': filepath,
            'path_id': count
        }
        print("feature", feature)
        # if(i==0):
        #     print("labeled_image", labeled_image, label, feature)
        #     plt.imshow(labeled_image)
        #     plt.axis('off')  # Turn off the axis labels
        #     plt.show()
        features.append(feature)
    return features



In [ ]:
#HUANGSfUZZY

def Huang(data):
    threshold=-1;
    first_bin=  0
    for ih in range(254):
        if data[ih] != 0:
            first_bin = ih
            break
    last_bin=254;
    for ih in range(254,-1,-1):
        if data[ih] != 0:
            last_bin = ih
            break
    term = 1.0 / (last_bin - first_bin)
    mu_0 = numpy.zeros(shape=(254,1))
    num_pix = 0.0
    sum_pix = 0.0
    for ih in range(first_bin,254):
        sum_pix = sum_pix + (ih * data[ih])
        num_pix = num_pix + data[ih]
        mu_0[ih] = sum_pix / num_pix
    min_ent = float("inf")
    for it in range(254):
        ent = 0.0
        for ih in range(it):
            mu_x = 1.0 / ( 1.0 + term * math.fabs( ih - mu_0[it]))
            if ( not ((mu_x  < 1e-06 ) or (mu_x > 0.999999))):
                ent = ent + data[ih] * (-mu_x * math.log(mu_x) - (1.0 - mu_x) * math.log(1.0 - mu_x) )
        if (ent < min_ent):
            min_ent = ent
            threshold = it
    return threshold

In [ ]:
#ROLLINGTHE BALL
def rolling_ball_background(array, radius, light_background=True, smoothing=True):
    ball = RollingBall(radius)
    float_array = array
    float_array = rolling_ball_float_background(float_array, radius, invert, smoothing, ball)
    background_pixels = float_array.flatten()
    pixels = numpy.int8(array.flatten())
    for p in range(len(pixels)):
        value = (pixels[p] & 0xff) - (background_pixels[p] + 255) + offset
        if value < 0:
            value = 0
        if value > 255:
            value = 255
        pixels[p] = numpy.int8(value)
    return numpy.reshape(pixels, array.shape)
def roll_ball(ball, array):
    height, width = array.shape
    pixels = numpy.float32(array.flatten())
    z_ball = ball.data
    ball_width = ball.width
    radius = ball_width / 2
    cache = numpy.zeros(width * ball_width)
    for y in range(-radius, height + radius):
        next_line_to_write_in_cache = (y + radius) % ball_width
        next_line_to_read = y + radius
        if next_line_to_read < height:
            src = next_line_to_read * width
            dest = next_line_to_write_in_cache * width
            cache[dest:dest + width] = pixels[src:src + width]
            p = next_line_to_read * width
            for x in range(width):
                pixels[p] =- float('inf')
                p += 1
        y_0 = y - radius
        if y_0 < 0:
            y_0 = 0
        y_ball_0 = y_0 - y + radius
        y_end = y + radius
        if y_end >= height:
            y_end = height - 1
        for x in range(-radius, width + radius):
            z = float('inf')
            x_0 = x - radius
            if x_0 < 0:
                x_0 = 0
            x_ball_0 = x_0 - x + radius
            x_end = x + radius
            if x_end >= width:
                x_end = width - 1
            y_ball = y_ball_0
            for yp in range(y_0, y_end + 1):
                cache_pointer = (yp % ball_width) * width + x_0
                bp = x_ball_0 + y_ball * ball_width
                for xp in range(x_0, x_end + 1):
                    z_reduced = cache[cache_pointer] - z_ball[bp]
                    if z > z_reduced:
                        z = z_reduced
                    cache_pointer += 1
                    bp += 1
                y_ball += 1
            y_ball = y_ball_0
            for yp in range(y_0, y_end + 1):
                p = x_0 + yp * width
                bp = x_ball_0 + y_ball * ball_width
                for xp in range(x_0, x_end + 1):
                    z_min = z + z_ball[bp]
                    if pixels[p] < z_min:
                        pixels[p] = z_min
                    p += 1
                    bp += 1
                y_ball += 1
    return numpy.reshape(pixels, array.shape)
class RollingBall(object):
    def __init__(self, radius):
        if radius <= 10:
            self.shrink_factor = 1
            arc_trim_per = 24
        self.build(radius, arc_trim_per)
    def build(self, ball_radius, arc_trim_per):
        small_ball_radius = ball_radius / self.shrink_factor
        if small_ball_radius < 1:
            small_ball_radius = 1
        rsquare = small_ball_radius * small_ball_radius
        xtrim = int(arc_trim_per * small_ball_radius) / 100
        half_width = int(round(small_ball_radius - xtrim))
        self.width = (2 * half_width) + 1
        self.data = [0.0] * (self.width * self.width)
        p = 0
        for y in range(self.width):
            for x in range(self.width):
                xval = x - half_width
                yval = y - half_width
                temp = rsquare - (xval * xval) - (yval * yval)

                if temp > 0:
                    self.data[p] = float(math.sqrt(temp))
                p += 1

In [ ]:
#MORPHOLOGICAL
input = 'MIAS6'
i = 0
start = time.time()
for img in glob.glob(input + '/1/*.png'):
    image = cv2.imread(img,0)
    kernel = np.ones((120,120),np.uint8)
    erosion = cv2.erode(image,kernel,iterations = 1)
    kernel = np.ones((120,120),np.uint8)
    dilation = cv2.dilate(erosion, kernel, iterations = 1)
    merged = cv2.bitwise_and(image, image , mask=dilation)
    i += 1
end = time.time()
print(end-start)

In [ ]:

# from segmentation import perform_segmentation, extract_features_from_segmentation
# from BackgroundRemoval.HuangsFuzzyThresholding import Huang
# import BackgroundRemoval.MorphologicalTransformation
# from BackgroundRemoval.RollingBallAlgorithm import RollingBall
# from extract_circular_points import extract_circular_points

# Define the image size
IMG_SIZE = 50

TRAIN_DIR = 'C:/Users/Kizzie/Desktop/PROJECT/Project-old_file/archive/Image_to_dataset/fiveTwelveSmall/train'
# TRAIN_DIR = 'C:/Users/Kizzie/Desktop/PROJECT/Project-old_file/archive/fiveTwelve/train'
TEST_DIR = 'C:/Users/Kizzie/Desktop/PROJECT/Project-old_file/archive/Image_to_dataset/fiveTwelveSmall/test'
# TEST_DIR = 'C:/Users/Kizzie/Desktop/PROJECT/Project-old_file/archive/fiveTwelve/test'
VAL_DIR = 'C:/Users/Kizzie/Desktop/PROJECT/Project-old_file/archive/Image_to_dataset/fiveTwelveSmall/val'
# VAL_DIR = 'C:/Users/Kizzie/Desktop/PROJECT/Project-old_file/archive/fiveTwelve/val'


# Define the labels for the two classes (normal and abnormal)
CATEGORIES = ['false', 'true']

def segmentation_with_image_show(labeled_image, img_gray):
    # Calculate the features from each segmented region
    regions = measure.regionprops(labeled_image, intensity_image=img_gray)

    circular_points = extract_circular_points(regions)

    # Create subplots for displaying images side by side
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    # Display the grayscale image without the red color
    axs[0].imshow(img_gray, cmap='gray')
    axs[0].set_title('')

    # Display the original image with circular points
    axs[1].imshow(img_gray)
    axs[1].set_title('')

    # Plot the circular points and perimeters
    for circular_point in circular_points:
        centroid_row = circular_point['centroid_row']
        centroid_col = circular_point['centroid_col']
        additional_results = circular_point['additional_results']

        # Plot the centroid point as a dot
        axs[1].plot(centroid_col, centroid_row, 'ro', markersize=5)

        # Plot the circular and elliptical perimeters
        for perimeter_row, perimeter_col in additional_results['perimeter_pixel_values']:
            axs[1].plot(perimeter_col, perimeter_row, 'r-', linewidth=1)

    # Adjust spacing between subplots
    plt.tight_layout()

    # Show the plot
    plt.show()

# Function to read and preprocess images
def read_and_preprocess_image(filepath):
    # Read the image file
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    # Resize the image to the desired size
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    # Flatten the image to a 1D array
    img = img.flatten()
    return img

# Function to load images and labels from a directory
def load_data_from_folder(folder):
    # Initialize the image and label lists
    images = []
    labels = []
    features = []
    folds = os.listdir(folder)
    trial = False
    # Loop over the categories
    for category in folds:
        # Define the path to the category folder
        path = os.path.join(folder, category)
        # Loop over the image files in the category folder
        for index, filename in enumerate(os.listdir(path)):
            # Define the path to the image file
            filepath = os.path.join(path, filename)
            # Read and preprocess the image
            img = read_and_preprocess_image(filepath)

            # Instantiate the RollingBall class
            radius = 10  # Example radius
            arc_trim_per = 24  # Example arc_trim_per
            rolling_ball = RollingBall(radius)

            # Call the build method
            rolling_ball.build(radius, arc_trim_per)

            # Access the properties and methods of the RollingBall object as needed
            width = rolling_ball.width
            data = rolling_ball.data

            print("width", width, "data", data)

            def apply_fuzzy_thresholding(image):
                # Apply Huang's Fuzzy Thresholding for background removal
                # Convert the image to grayscale
                if len(image.shape) == 3:
                    # Convert the image to grayscale
                    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                else:
                    gray = image

                # Apply Huang's Fuzzy Thresholding
                _, thresholded = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

                # Perform morphological operations to fill holes and remove noise
                kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
                closed = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)

                # Create a mask of the foreground regions
                mask = cv2.bitwise_and(image, image, mask=closed)

                # Return the masked image
                return mask

            # Background removal
            # Load the grayscale image
            # Append the image and label to the lists
            label = 1
            if "false" in category:
                label = 0
            labeled_image = perform_segmentation(filepath)
            img_gray = color.rgb2gray(io.imread(filepath))
            img_gray_1 = color.rgb2gray(io.imread(filepath))

            if(index == 0 and category == folds[0]):
                print("trail", trial, folds, path, index)
                filepath1 = "C:/Users/Kizzie/Desktop/PROJECT/Project-old_file/archive/fiveTwelve/train/train_l_mlo_true/39672040.png"
                print("Processing file {}".format(filepath1))

                img = read_and_preprocess_image(filepath1)

                # Load the grayscale image
                # Append the image and label to the lists
                labeled_image = perform_segmentation(filepath1)
                img_gray = color.rgb2gray(io.imread(filepath1))

                feature_array = segmentation_with_image_show(labeled_image, img_gray)

                print(feature_array)

                img_gray_1 = color.rgb2gray(io.imread(filepath1))
                image = img_gray_1
                if image is None:
                    raise ValueError(f"Failed to load image from .")

                # Remove background using Huang's Fuzzy Thresholding
                removed_bg_fuzzy_thresholding = apply_fuzzy_thresholding(image)

                # Display the original and processed images
                # Display the original and processed images
                plt.figure(figsize=(12, 6))

                # Convert BGR image to RGB for proper display with plt
                rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                plt.subplot(1, 2, 1)
                plt.imshow(rgb_image)
                plt.title("")
                plt.axis('off')

                plt.subplot(1, 2, 2)
                plt.imshow(removed_bg_fuzzy_thresholding, cmap='gray')
                plt.title("")
                plt.axis('off')

                plt.tight_layout()
                plt.show()
            # image = img_gray_1
            # if image is None:
            #     raise ValueError(f"Failed to load image from .")

            # # image_with_light_bg = create_light_background(removed_bg_rolling_ball)

            # # Remove background using Huang's Fuzzy Thresholding
            # removed_bg_fuzzy_thresholding = apply_fuzzy_thresholding(image)

            # # Display the original and processed images
            # # Display the original and processed images
            # plt.figure(figsize=(12, 6))

            # # Convert BGR image to RGB for proper display with plt
            # rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # plt.subplot(1, 2, 1)
            # plt.imshow(rgb_image)
            # plt.title("")
            # plt.axis('off')

            # plt.subplot(1, 2, 2)
            # plt.imshow(removed_bg_fuzzy_thresholding, cmap='gray')
            # plt.title("")
            # plt.axis('off')

            # plt.tight_layout()
            # plt.show()

            feature_array = extract_features_from_segmentation(labeled_image, img_gray, label, filepath)
            print("featire_array", feature_array)
            # fieldnames = list(feature_array[0].keys())
            # print("fieldNames", fieldnames)
            if(len(feature_array)>0):
                features.append(feature_array[0])
            for feat in feature_array:
                if(feat['radius']<500):
                    features.append(feature_array[0])
            images.append(img)
            labels.append(label)
            # feature["label"] = label
            # feature["path"] = filepath
            # print("label", feature)
    print("features", len(features))
    return images, labels, features

# Load the train, test, and val data
train_images, train_labels, train_feature = load_data_from_folder(TRAIN_DIR)
test_images, test_labels, test_feature = load_data_from_folder(TEST_DIR)
val_images, val_labels, val_feature = load_data_from_folder(VAL_DIR)

data = train_feature
# Open a CSV file for writing
with open('train_final1.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Write the header row
    writer.writerow(data[0].keys())

    # Write each row of data
    for row in data:
        writer.writerow(row.values())


